In [5]:
import numpy as np
np.random.seed(42)

dataset

In [6]:
# XOR:
xor_gate = np.array([[0, 0, 0], [0, 1, 1], [1, 0, 1], [1, 1, 0]])

# AND
and_gate = np.array([[1, 1, 1], [0, 1, 0], [1, 0, 0], [0, 0, 0]])

# IRIS 3 especies
# columnas = [largo de sepalo, ancho de petalo] 0=setosa , 1=versicolor, 2 = virginica
iris = np.array([[4.7, 0.2, 0], [4.6, 0.2, 0], [7, 1.4, 1], [6.4, 1.5, 1], [6.3, 1.8, 2], [6.5, 2.2, 2]])

# creamos una entrada
X = np.array([[0], [1]])
Y = np.array([[1]])

funciones

In [7]:
def sigmoid(x, derived=False):
  if derived:
    return sigmoid(x)*(1 - sigmoid(x))
  else:
    return 1/(1+np.exp(-x))
  
def relu(x, derived=False):
  if derived:
    return 0 if x <= 0 else x
  else:
    return np.maximum(0, x)

def MSE(y_true, y_pred, derived=False):
  if derived:
    return y_pred - y_true
  else:
    return np.square(y_true - y_pred).sum()/2

# Paso a paso

Perceptron multicapa

In [8]:
# tamaño de las capas
n_input, n_hidden, n_out = 2, 2, 1

# coeficiente de aprendizaje
alfa = 0.25

# inicializar pesos random dentro del rango de una distribucion normal
w_hidden = np.random.rand(2, 2)
b_hidden = np.random.rand(2, 1)
w_out = np.random.rand(1, 2)
b_out = np.random.rand(1, 1)

# guardar error
error = []

Propagar

In [9]:
# operar en la capa oculta
sum_hidden = w_hidden.dot(X) + b_hidden
out_hidden = sigmoid(sum_hidden)
out_hidden

array([[0.75151953],
       [0.68019172]])

In [10]:
# operar en la capa de salida
sum_out = w_out.dot(out_hidden) + b_out
out_out = sigmoid(sum_out)
out_out

array([[0.77450599]])

Calcular el ERROR de la red neuronal

In [11]:
error.append(MSE(Y, out_out))
MSE(Y, out_out)

0.025423773331154486

Retropropagamos

In [12]:
# operamos en la capa de salida OUT

# calculamos el error inputado a cada neurona
dC_da_out = MSE(Y, out_out, derived=True)
dA_dz_out = sigmoid(sum_out, derived=True)
delta_out = dC_da_out * dA_dz_out
print(delta_out)

# calculamos el error inputado a cada parametro
dZ_dw_out = out_hidden
dC_dw_out = delta_out.dot(dZ_dw_out.T)

dZ_db_out = 1
dC_db_out = delta_out # delta x 1 = delta

# actualizamos el valor de los pesos
w_out_new = w_out - alfa * dC_dw_out
b_out_new = b_out - alfa * dC_db_out
w_out_new, b_out_new

[[-0.03938173]]


(array([[0.06548265, 0.87287293]]), array([[0.61096044]]))

In [13]:
# operamos en la capa oculta HIDDEN

# retropropagamos el error calculado en la capa de salida
delta_hidden = w_out.T.dot(delta_out) * sigmoid(sum_hidden, derived=True)
print(delta_hidden)

# calculamos el error inputado a cada parametro
dZ_dw_hidden = X
dC_dw_hidden = delta_hidden.dot(dZ_dw_hidden.T)

dZ_db_hidden = 1
dC_db_hidden = delta_hidden

# # actualizamos el valor de los pesos
w_hidden_new = w_hidden - alfa * dC_dw_hidden
b_hidden_new = b_hidden - alfa * dC_db_hidden
w_hidden_new, b_hidden_new

[[-0.00042715]
 [-0.00742031]]


(array([[0.37454012, 0.95082109],
        [0.73199394, 0.60051356]]),
 array([[0.15612543],
        [0.1578496 ]]))

PROPAGAR y Calcular ERROR

In [14]:
# operar en la capa oculta
sum_hidden_new = w_hidden_new.dot(X)
out_hidden_new = sigmoid(sum_hidden_new)

# operar en la capa de salida
sum_out_new = w_out_new.dot(out_hidden_new)
out_out_new = sigmoid(sum_out_new)
print(out_out_new)

# calcular error
error.append(MSE(Y, out_out_new))

[[0.64814831]]


In [15]:
# el error disminuye
error

[0.025423773331154486, 0.061899805065811705]

# Implementacion

In [ ]:
# def __init__(self, layers):
#     self.layers = layers
#     self.errores = []
#     # pesos iniciales W
#     if layers == [2, 3, 3]:
#         self.W = [np.array([[0.1, 0.8], [0.7, 0.9], [0.5, 0.3]]), 
#                   np.array([[0.3, 0.7, 0.5], [0.5, 0.1, 0.5], [0.2, 0.8, 0.9]])]
#         self.b = [np.array([[0.4], [0.5], [0.8]]), np.array([[0.5], [0.9], [0.7]])]
#     else:
#         self.W = []
#         self.b = []
#         for i in range(len(layers)-1):
#             self.W.append(np.random.normal(0, 0.1, size=(layers[i+1], layers[i])))
#             self.b.append(np.random.normal(0, 0.1, size=(layers[i+1], 1)))

In [21]:
class RNA:
    # constructor
    def __init__(self, layers):
        self.layers = layers
        self.errores = []
        self.W, self.b = [], []
        for i in range(len(layers)-1):
            self.W.append(np.random.normal(0, 0.1, size=(layers[i+1], layers[i])))
            self.b.append(np.random.normal(0, 0.1, size=(layers[i+1], 1)))
    
    def propagar(self, X):
        Sum = [np.array([]) for _ in range(len(self.layers)-1)]
        Out = [np.array([]) for _ in range(len(self.layers)-1)]
        for i in range(len(self.layers)-1):
            input = X if i == 0 else Out[i-1]
            Sum[i] = self.W[i].dot(input) + self.b[i]
            Out[i] = sigmoid(Sum[i])
        return Sum, Out
    
    def retropropagacion(self, X, Y, alfa, Sum, Out):
        delta = [np.array([]) for _ in range(len(self.layers) - 1)]
        W_old = self.W.copy()
        for i in range(len(self.layers)-1):
            j = (i+1)*(-1) # itera -1, -2, ...
            if j == -1:
                delta[j] = MSE(Y, Out[j], derived=True) * sigmoid(Sum[j], derived=True)
            else:
                delta[j] = W_old[j+1].T.dot(delta[j+1]) * sigmoid(Sum[j], derived=True)

            # si esta en la primera capa oculta la dZ_dw es el input X
            dZ_dw = X if i == len(self.layers)-2 else Out[j-1]

            # acualizar los pesos y sesgos
            self.W[j] = self.W[j] - alfa * delta[j].dot(dZ_dw.T)
            self.b[j] = self.b[j] - alfa * delta[j]

    def fit(self, X, Y, alfa, epoch):
        for _ in range(epoch):
            for x, y in zip(X, Y):
                Sum, Out = self.propagar(x)
                self.errores.append(MSE(y, Out[-1]))
                self.retropropagacion(x, y, alfa, Sum, Out)

    def prediction(self, X):
        _, Out = self.propagar(X)
        return Out[-1]
   

In [22]:
salidas = [row[-1].reshape((1, 1)) for row in xor_gate]
entradas = [row[:-1].reshape((2, 1)) for row in xor_gate]

In [23]:
rna = RNA([2, 2, 1])
rna.fit(entradas[1:2], salidas[1:2], 0.01, 10)

In [24]:
rna.errores[-5:]

[0.13458878043789332,
 0.13434857100374148,
 0.13410876180789685,
 0.133869353020294,
 0.13363034480453856]

In [309]:
rna.prediction(X)

array([[0.50191213]])